In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.autograd import Variable
import numpy as np


# Universal import block 
# Block to get the relative imports working 
import os, sys, re
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

import config
import matplotlib.pyplot as plt 
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap
import adversarial_attacks_refactor as aar 
import spatial_transformers as st 
import importlib
importlib.reload(ap)

### START pytorch_image_classification imports
import time, random, json, logging, argparse, csv
# import numpy as np
# import torch.optim
# import torch.utils.data
# import torch.backends.cudnn
# import torchvision.utils
from pytorch_image_classification_dataloader_c10h import get_loader
from pytorch_image_classification_utils import (str2bool, load_model, save_checkpoint, create_optimizer,
                                                AverageMeter, mixup, CrossEntropyLoss, onehot)
# from rutils_run import save_checkpoint_epoch
from pytorch_image_classification_argparser import get_config

use_gpu = False

In [29]:
sys.argv = ['']

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--arch', type=str, default='resnet')
    parser.add_argument('--config', type=str, default='tmp_reference_model/resnet_basic_110_config.json')
    # model config (VGG)
    parser.add_argument('--n_channels', type=str)
    parser.add_argument('--n_layers', type=str)
    parser.add_argument('--use_bn', type=str2bool)
    #
    parser.add_argument('--base_channels', type=int)
    parser.add_argument('--block_type', type=str)
    parser.add_argument('--depth', type=int)
    # model config (ResNet-preact)
    parser.add_argument('--remove_first_relu', type=str2bool)
    parser.add_argument('--add_last_bn', type=str2bool)
    parser.add_argument('--preact_stage', type=str)
    # model config (WRN)
    parser.add_argument('--widening_factor', type=int)
    # model config (DenseNet)
    parser.add_argument('--growth_rate', type=int)
    parser.add_argument('--compression_rate', type=float)
    # model config (WRN, DenseNet)
    parser.add_argument('--drop_rate', type=float)
    # model config (PyramidNet)
    parser.add_argument('--pyramid_alpha', type=int)
    # model config (ResNeXt)
    parser.add_argument('--cardinality', type=int)
    # model config (shake-shake)
    parser.add_argument('--shake_forward', type=str2bool)
    parser.add_argument('--shake_backward', type=str2bool)
    parser.add_argument('--shake_image', type=str2bool)
    # model config (SENet)
    parser.add_argument('--se_reduction', type=int)

    parser.add_argument('--outdir', type=str, required=False)
    parser.add_argument('--seed', type=int, default=17)
    parser.add_argument('--test_first', type=str2bool, default=True)
    parser.add_argument('--gpu', type=str, default='0') # -1 for CPU
    # TensorBoard configuration
    parser.add_argument(
        '--tensorboard', dest='tensorboard', action='store_true', default=True)
    parser.add_argument(
        '--no-tensorboard', dest='tensorboard', action='store_false')
    parser.add_argument('--tensorboard_train_images', action='store_true')
    parser.add_argument('--tensorboard_test_images', action='store_true')
    parser.add_argument('--tensorboard_model_params', action='store_true')
    # configuration of optimizer
    parser.add_argument('--epochs', type=int)
    parser.add_argument('--batch_size', type=int)
    parser.add_argument('--optimizer', type=str, choices=['sgd', 'adam'])
    parser.add_argument('--base_lr', type=float)
    parser.add_argument('--weight_decay', type=float)
    # configuration for SGD
    parser.add_argument('--momentum', type=float)
    parser.add_argument('--nesterov', type=str2bool)
    # configuration for learning rate scheduler
    parser.add_argument(
        '--scheduler', type=str, choices=['none', 'multistep', 'cosine'])
    # configuration for multi-step scheduler]
    parser.add_argument('--milestones', type=str)
    parser.add_argument('--lr_decay', type=float)
    # configuration for cosine-annealing scheduler]
    parser.add_argument('--lr_min', type=float, default=0)
    # configuration for Adam
    parser.add_argument('--betas', type=str)
    # configuration of data loader
    parser.add_argument(
        '--dataset',
        type=str,
        default='CIFAR10',
        choices=['CIFAR10', 'CIFAR10H'])
    parser.add_argument('--num_workers', type=int, default=7)
    # cutout configuration
    parser.add_argument('--use_cutout', action='store_true', default=False)
    parser.add_argument('--cutout_size', type=int, default=16)
    parser.add_argument('--cutout_prob', type=float, default=1)
    parser.add_argument('--cutout_inside', action='store_true', default=False)
    # random erasing configuration
    parser.add_argument(
        '--use_random_erasing', action='store_true', default=False)
    parser.add_argument('--random_erasing_prob', type=float, default=0.5)
    parser.add_argument(
        '--random_erasing_area_ratio_range', type=str, default='[0.02, 0.4]')
    parser.add_argument(
        '--random_erasing_min_aspect_ratio', type=float, default=0.3)
    parser.add_argument('--random_erasing_max_attempt', type=int, default=20)
    # mixup configuration
    parser.add_argument('--use_mixup', action='store_true', default=False)
    parser.add_argument('--mixup_alpha', type=float, default=1)

    # previous model weights to load if any
    parser.add_argument('--resume', type=str)
    # whether to tune to human labels
    parser.add_argument('--human_tune', action='store_true', default=False)
    # where to save the loss/accuracy for c10h to a csv file
    parser.add_argument('--c10h_scores_outdir', type=str, default='tmp')
    # c10h scores save interval (in epochs)
    parser.add_argument('--c10h_save_interval', type=str, default='1') # changed from int
    # how much of the data to use use for test for c10h training
    parser.add_argument('--c10h_testsplit_percent', type=float, default=0.1)
    # seed for splitting the c10h data into train/test
    parser.add_argument('--c10h_datasplit_seed', type=int, default=999)
    # whether to use the cifar10 labels for the human test set (CONTROL)
    parser.add_argument('--nonhuman_control', type=str2bool, default=False)
    # whether to sample from the human labels to get one-hot samples
    parser.add_argument('--c10h_sample', action='store_true', default=False)
    # whether to save to out_dir
    parser.add_argument('--no_output', action='store_true', default=False)
    # to test the loaded model and don't train
    parser.add_argument('--test_only', action='store_true', default=False)

    args = parser.parse_args()
    # if not is_tensorboard_available:
    args.tensorboard = False

    config = get_config(args)

    return config

config = parse_args()

run_config = config['run_config']

run_config['resume'] = 'tmp_reference_model/model_best_state_resnet_basic_110_con_False_lr_001_seed_0.pth'
#run_config['resume'] = 'tmp_reference_model/model_best_state.pth'

def load_our_model(config, weights_path):
        
    our_model = load_model(config['model_config'])
    
    # load pretrained weights if given
    if os.path.isfile(weights_path):
        print("=> loading checkpoint '{}'".format(weights_path))

        # Resolve CPU/GPU stuff
        if use_gpu:
            map_location = None
        else:
            map_location= (lambda s, l: s)

        checkpoint = torch.load(weights_path,
                                map_location=map_location)

        correct_state_dict = {re.sub(r'^module\.', '', k): v for k, v in
                              checkpoint['state_dict'].items()}

        our_model.load_state_dict(correct_state_dict)

        print("=> loaded checkpoint '{}' (epoch {})"
              .format(weights_path, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(weights_path))
        
    return our_model

model = load_our_model(config, run_config['resume'])

=> loading checkpoint 'tmp_reference_model/model_best_state_resnet_basic_110_con_False_lr_001_seed_0.pth'
=> loaded checkpoint 'tmp_reference_model/model_best_state_resnet_basic_110_con_False_lr_001_seed_0.pth' (epoch 72)


In [30]:
""" Goal here is to make sure adversarial training works under the refactored model. It _should_, but it might need 
    a few tweaks 
"""

# Load up dataLoader, classifier, normer 
# use_gpu = torch.cuda.is_available()
# classifier_net = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
#                                                            use_gpu=use_gpu)

classifier_net = load_our_model(config, run_config['resume'])

classifier_net.eval()
train_loader = cifar_loader.load_cifar_data('train', normalize=False, 
                                            batch_size=16, use_gpu=use_gpu)
val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          batch_size=4, use_gpu=use_gpu)

cifar_normer = utils.DifferentiableNormalize(mean=np.array([0.4914, 0.4822, 0.4465]),
                                             std=np.array([0.2470, 0.2435, 0.2616]))

examples, labels = next(iter(val_loader))

=> loading checkpoint 'tmp_reference_model/model_best_state_resnet_basic_110_con_False_lr_001_seed_0.pth'
=> loaded checkpoint 'tmp_reference_model/model_best_state_resnet_basic_110_con_False_lr_001_seed_0.pth' (epoch 72)
Files already downloaded and verified
Files already downloaded and verified


In [31]:
# Make a threat model and attack object 
importlib.reload(advtrain)
importlib.reload(aar)
delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                              ap.PerturbationParameters(lp_style='inf',
                                                        lp_bound=8.0 / 255.0))
loss_fxn = plf.VanillaXentropy(classifier_net, normalizer=cifar_normer) # USE A PLF LOSS FXN 
fgsm_attack = aar.FGSM(classifier_net, cifar_normer, delta_threat, loss_fxn)
attack_params = advtrain.AdversarialAttackParameters(fgsm_attack, 0.5, 
                                                     attack_specific_params={'attack_kwargs': {'step_size': 0.1}})
print(attack_params)

rot_threat = ap.ThreatModel(ap.ParameterizedXformAdv, 
                            ap.PerturbationParameters(lp_style='inf', 
                                                      lp_bound=10.0 / 360,
                                                      xform_class=st.RotationTransform))
loss_fxn_rot = plf.VanillaXentropy(classifier_net, normalizer=cifar_normer) # USE A PLF LOSS FXN 
rot_attack = aar.PGD(classifier_net, cifar_normer, rot_threat, loss_fxn_rot)
rot_attack_params = advtrain.AdversarialAttackParameters(rot_attack, 1.0, 
                                                     attack_specific_params={'attack_kwargs': {'step_size': 0.01}})

In [16]:
print(isinstance(attack_params, advtrain.AdversarialAttackParameters))

True


In [21]:
importlib.reload(advtrain)
            
classifier_net.train()
train_obj = advtrain.AdversarialTraining(classifier_net, cifar_normer, 'refactor_test', 'shake_shake')
train_loss = nn.CrossEntropyLoss() # USE A PREBUILT TRAIN LOSS FXN 
train_obj.train(train_loader, 2, train_loss, attack_parameters=[attack_params], verbosity="snoop", 
                adversarial_save_dir='shake_shake', regularize_adv_scale=1.0)

(Post FGSM):  25.0 correct
[1,     1] accuracy: (25.000, 75.000)


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7ff8cda93358>>
Traceback (most recent call last):
  File "/home/joshuacp/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/joshuacp/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/joshuacp/anaconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/joshuacp/anaconda3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/joshuacp/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/joshuacp/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py

FileNotFoundError: [Errno 2] No such file or directory: '/tigress/joshuacp/projects/cifar10-human-experiments/mister_ed/output_images/shake_shake/3823205652074535.examples.npy'